In [45]:
import pandas as pd
import numpy as np

In [46]:
from os import listdir
from os.path import isfile, join

In [47]:
from os import listdir
from os.path import isfile, join

file_paths = [r"C:\Users\dell 1\Desktop\Prediction-of-Hard-Drive-Failure-master\short_data_Q1"]

files = []

for filepath in file_paths:
    files_1 = [f for f in listdir(filepath) if isfile(join(filepath, f))]
    files_1 = [join(filepath, f) for f in files_1 if f.endswith(".csv") and f.startswith("202")]
    files.extend(files_1)

len(files)


10

# SET MODEL NUMBER HERE

In [48]:
# set the model name here 
model_name = "ST12000NM0007"
# model_name = "ST4000DM000"
# model_name = "ST8000NM0055"
# model_name = "ST8000DM002"
# model_name = "TOSHIBA MQ01ABF050"
# model_name = "ST500LM012 HN"
# model_name = "HGST HMS5C4040BLE640"

# RAW DATA

In [49]:
# put all the dataframes into a list 
master_df_list = []

data = pd.read_csv(files[0])
data.shape

(235597, 179)

In [50]:
# determine all the columns that have to be dropped by default
def drop_column_names():
    data = pd.read_csv(files[0])
    drop_columns = []
    for i in data.columns:
        if "normalized" in i:
            drop_columns.append(i)     
    return drop_columns

In [51]:
# columns that have to be dropped
drop_names = drop_column_names()

In [52]:
# iterate over each file and drop the unwanted columns
for file_name in files:
    data = pd.read_csv(file_name)
    data.drop(drop_names, axis = 1, inplace=True)
    data = data[data["model"] == model_name]
    master_df_list.append(data.copy())
    print("{} added to master_list".format(file_name))

C:\Users\dell 1\Desktop\Prediction-of-Hard-Drive-Failure-master\short_data_Q1\2023-01-01.csv added to master_list
C:\Users\dell 1\Desktop\Prediction-of-Hard-Drive-Failure-master\short_data_Q1\2023-01-02.csv added to master_list
C:\Users\dell 1\Desktop\Prediction-of-Hard-Drive-Failure-master\short_data_Q1\2023-01-03.csv added to master_list
C:\Users\dell 1\Desktop\Prediction-of-Hard-Drive-Failure-master\short_data_Q1\2023-01-04.csv added to master_list
C:\Users\dell 1\Desktop\Prediction-of-Hard-Drive-Failure-master\short_data_Q1\2023-01-05.csv added to master_list
C:\Users\dell 1\Desktop\Prediction-of-Hard-Drive-Failure-master\short_data_Q1\2023-01-06.csv added to master_list
C:\Users\dell 1\Desktop\Prediction-of-Hard-Drive-Failure-master\short_data_Q1\2023-01-07.csv added to master_list
C:\Users\dell 1\Desktop\Prediction-of-Hard-Drive-Failure-master\short_data_Q1\2023-01-08.csv added to master_list
C:\Users\dell 1\Desktop\Prediction-of-Hard-Drive-Failure-master\short_data_Q1\2023-01-09

In [53]:
# generate a master data frame to have all the data
print("concat started")
master_df = pd.concat(master_df_list, sort=False)
print("concat complete")
master_df.shape
print("{} ::: original column count : {}".format(model_name, master_df.shape[1]))

concat started
concat complete
ST8000NM0055 ::: original column count : 92


In [54]:
# data cleaning operations
def data_cleaning(model_name):
    # model_1 = master_df[master_df["model"] == model_name]
    
    # drop all columns that are only NaN
    # model_1 = model_1.dropna(axis=1, how='all')
    print("function execution has begun")
    model_1 = master_df.dropna(axis=1, how='all')
    print("initial column drop done. Shape : {}".format(model_1.shape))
    print("{} ::: Dropping NaN Value Columns : {}".format(model_name, model_1.shape[1]))
    
    # drops all rows with even one NaN value
    model_1 = model_1.dropna()
    print("initial row drop done. Shape : {}".format(model_1.shape))
    
    # find all columns that are filled with the same value and drop them
    columns_with_same_values = model_1.columns[model_1.nunique() <= 1].tolist()
    
    # do not remove the columns 'model' and 'capacity_bytes'
    columns_with_same_values.remove('model')
    columns_with_same_values.remove('capacity_bytes')
    model_1 = model_1.drop(columns_with_same_values, axis=1)
    print("repeating cloumn value drop done. Shape : {}".format(model_1.shape))
    print("{} ::: Dropping Columns With Same Value: {}".format(model_name, model_1.shape[1]))
    
    # sort the dataframe based on serial_number and date
    model_1 = model_1.sort_values(['serial_number', 'date'], ascending=[True, True])
    print("sort done. Shape : {}".format(model_1.shape))
    return model_1

In [55]:
# cleaned data
cleaned_data = data_cleaning(model_name)
print("cleaned_data obtained")

function execution has begun
initial column drop done. Shape : (144151, 29)
ST8000NM0055 ::: Dropping NaN Value Columns : 29
initial row drop done. Shape : (144087, 29)
repeating cloumn value drop done. Shape : (144087, 25)
ST8000NM0055 ::: Dropping Columns With Same Value: 25
sort done. Shape : (144087, 25)
cleaned_data obtained


In [56]:
last_day_data = cleaned_data.groupby('serial_number').tail(1)
export_filename = r"C:\Users\dell 1\Desktop\Prediction-of-Hard-Drive-Failure-master\dataset2\\" + model_name + "_last_day_all_q_raw.csv"
last_day_data.to_csv(export_filename, index=False)
print("{} ::: export complete. Shape : {}".format(export_filename, last_day_data.shape))


C:\Users\dell 1\Desktop\Prediction-of-Hard-Drive-Failure-master\dataset2\\ST8000NM0055_last_day_all_q_raw.csv ::: export complete. Shape : (14427, 25)


In [57]:
n = 10  # number of days data needed
last_n_days_data = cleaned_data.groupby('serial_number').tail(n)

print("Before")
print(last_n_days_data["failure"].value_counts())

for index, row in last_day_data.iterrows():
    if row["failure"] == 1:
        last_n_days_data["failure"].mask(last_n_days_data["serial_number"] == row["serial_number"], 1, inplace=True)

print("\nAfter")
print(last_n_days_data["failure"].value_counts())

export_filepath = r"C:\Users\dell 1\Desktop\Prediction-of-Hard-Drive-Failure-master\dataset2\\" + model_name + "_last_" + str(n) + "_day_all_q_raw.csv"
last_n_days_data.to_csv(export_filepath, index=False)
print("{} ::: export complete. Shape : {}".format(export_filepath, last_n_days_data.shape))


Before
failure
0    144075
1        12
Name: count, dtype: int64


C:\Users\dell 1\AppData\Local\Temp\ipykernel_12456\1135245351.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  last_n_days_data["failure"].mask(last_n_days_data["serial_number"] == row["serial_number"], 1, inplace=True)
C:\Users\dell 1\AppData\Local\Temp\ipykernel_12456\1135245351.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on 


After
failure
0    144023
1        64
Name: count, dtype: int64
C:\Users\dell 1\Desktop\Prediction-of-Hard-Drive-Failure-master\dataset2\\ST8000NM0055_last_10_day_all_q_raw.csv ::: export complete. Shape : (144087, 25)


# NORMALISED DATA

In [58]:
# put all the dataframes into a list 
master_df_list = []

In [59]:
# determine all the columns that have to be dropped by default
def drop_column_names():
    data = pd.read_csv(files[0])
    drop_columns = []
    for i in data.columns:
        if "raw" in i:
            drop_columns.append(i)     
    return drop_columns

In [60]:
# columns that have to be dropped
drop_names = drop_column_names()

In [61]:
# iterate over each file and drop the unwanted columns
for file_name in files:
    data = pd.read_csv(file_name)
    data.drop(drop_names, axis = 1, inplace=True)
    data = data[data["model"] == model_name]
    master_df_list.append(data.copy())
    print("{} added to master_list".format(file_name))

C:\Users\dell 1\Desktop\Prediction-of-Hard-Drive-Failure-master\short_data_Q1\2023-01-01.csv added to master_list
C:\Users\dell 1\Desktop\Prediction-of-Hard-Drive-Failure-master\short_data_Q1\2023-01-02.csv added to master_list
C:\Users\dell 1\Desktop\Prediction-of-Hard-Drive-Failure-master\short_data_Q1\2023-01-03.csv added to master_list
C:\Users\dell 1\Desktop\Prediction-of-Hard-Drive-Failure-master\short_data_Q1\2023-01-04.csv added to master_list
C:\Users\dell 1\Desktop\Prediction-of-Hard-Drive-Failure-master\short_data_Q1\2023-01-05.csv added to master_list
C:\Users\dell 1\Desktop\Prediction-of-Hard-Drive-Failure-master\short_data_Q1\2023-01-06.csv added to master_list
C:\Users\dell 1\Desktop\Prediction-of-Hard-Drive-Failure-master\short_data_Q1\2023-01-07.csv added to master_list
C:\Users\dell 1\Desktop\Prediction-of-Hard-Drive-Failure-master\short_data_Q1\2023-01-08.csv added to master_list
C:\Users\dell 1\Desktop\Prediction-of-Hard-Drive-Failure-master\short_data_Q1\2023-01-09

In [62]:
# generate a master data frame to have all the data
print("concat started")
master_df = pd.concat(master_df_list, sort=False)
print("concat complete")
master_df.shape

concat started
concat complete


(144151, 92)

In [63]:
# data cleaning operations
def data_cleaning_normalised(model_name):
    # model_1 = master_df[master_df["model"] == model_name]
    
    # drop all columns that are only NaN
    print("function execution has begun")
    model_1 = master_df.dropna(axis=1, how='all')
    # model_1 = model_1.dropna(axis=1, how='all')
    print("initial column drop done. Shape : {}".format(model_1.shape))
    
    # drops all rows with even one NaN value
    model_1 = model_1.dropna()
    print("initial row drop done. Shape : {}".format(model_1.shape))
    
    # find all columns that are filled with the same value and drop them
    columns_with_same_values = model_1.columns[model_1.nunique() <= 1].tolist()
    
    # do not remove the columns 'model' and 'capacity_bytes'
    columns_with_same_values.remove('model')
    columns_with_same_values.remove('capacity_bytes')
    model_1 = model_1.drop(columns_with_same_values, axis=1)
    print("repeating cloumn value drop done. Shape : {}".format(model_1.shape))
    
    # sort the dataframe based on serial_number and date
    model_1 = model_1.sort_values(['serial_number', 'date'], ascending=[True, True])
    print("sort done. Shape : {}".format(model_1.shape))
    return model_1

In [64]:
# cleaned data
cleaned_data = data_cleaning_normalised(model_name)
print("cleaned_data obtained")

function execution has begun
initial column drop done. Shape : (144151, 29)
initial row drop done. Shape : (144087, 29)
repeating cloumn value drop done. Shape : (144087, 17)
sort done. Shape : (144087, 17)
cleaned_data obtained


In [65]:
last_day_data = cleaned_data.groupby('serial_number').tail(1)
export_filename = r"C:\Users\dell 1\Desktop\Prediction-of-Hard-Drive-Failure-master\dataset2\\" + model_name + "_last_day_all_q_normalised.csv"
last_day_data.to_csv(export_filename, index=False)
print("{} ::: export complete. Shape : {}".format(export_filename, last_day_data.shape))


C:\Users\dell 1\Desktop\Prediction-of-Hard-Drive-Failure-master\dataset2\\ST8000NM0055_last_day_all_q_normalised.csv ::: export complete. Shape : (14427, 17)


In [66]:
n = 10  # number of days data needed
last_n_days_data = cleaned_data.groupby('serial_number').tail(n)

print("Before")
print(last_n_days_data["failure"].value_counts())

for index, row in last_day_data.iterrows():
    if row["failure"] == 1:
        last_n_days_data["failure"].mask(last_n_days_data["serial_number"] == row["serial_number"], 1, inplace=True)

print("\nAfter")
print(last_n_days_data["failure"].value_counts())

export_filepath = r"C:\Users\dell 1\Desktop\Prediction-of-Hard-Drive-Failure-master\dataset2\\" + model_name + "_last_" + str(n) + "_day_all_q_normalised.csv"
last_n_days_data.to_csv(export_filepath, index=False)
print("{} ::: export complete. Shape : {}".format(export_filepath, last_n_days_data.shape))


Before
failure
0    144075
1        12
Name: count, dtype: int64


C:\Users\dell 1\AppData\Local\Temp\ipykernel_12456\1542373976.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  last_n_days_data["failure"].mask(last_n_days_data["serial_number"] == row["serial_number"], 1, inplace=True)
C:\Users\dell 1\AppData\Local\Temp\ipykernel_12456\1542373976.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on 


After
failure
0    144023
1        64
Name: count, dtype: int64
C:\Users\dell 1\Desktop\Prediction-of-Hard-Drive-Failure-master\dataset2\\ST8000NM0055_last_10_day_all_q_normalised.csv ::: export complete. Shape : (144087, 17)
